In [7]:
import pandas as pd
import os
from qdrant_client import QdrantClient, models
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from tqdm.auto import tqdm # Para barras de progresso visuais
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env")


QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
COLLECTION_NAME = "med_chat" # Novo nome de coleção para evitar conflitos
CSV_FILE_PATH = "C:\\Users\\drmar\\OneDrive\\Documentos\\code\\adk\\Triagem_Bot\\data\\mtsamples.csv"
TEXT_COLUMN_TO_EMBED = "descricao" # A coluna que será transformada em vetor


In [2]:
client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)

# Inicializa o modelo de embedding do Google via LangChain
print("Inicializando o modelo de embedding do Google...")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

# O modelo 'embedding-001' do Google gera vetores de 768 dimensões.
vector_size = 768
print(f"O modelo 'embedding-001' gera vetores de dimensão: {vector_size}")



Inicializando o modelo de embedding do Google...
O modelo 'embedding-001' gera vetores de dimensão: 768


In [ ]:
print(f"Criando a coleção '{COLLECTION_NAME}'...")
try:
    client.recreate_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(size=vector_size, distance=models.Distance.COSINE),
    )
    print("Coleção criada com sucesso!")
except Exception as e:
    print(f"A coleção já existe ou ocorreu um erro: {e}")

In [ ]:
# --- PASSO 3: LER O CSV, GERAR EMBEDDINGS E FAZER O UPLOAD ---

print("Lendo o arquivo CSV...")
df = pd.read_csv(CSV_FILE_PATH)

# Extrai a coluna de texto para gerar os embeddings em lote
texts_to_embed = df[TEXT_COLUMN_TO_EMBED].tolist()
print(f"Gerando embeddings para {len(texts_to_embed)} textos. Isso pode levar um momento...")

# Gera todos os embeddings de uma vez (mais eficiente)
vectors = embeddings.embed_documents(texts_to_embed)
print("Embeddings gerados com sucesso!")

# Converte o DataFrame para uma lista de dicionários para o payload
payloads = df.to_dict('records')
# Extrai os IDs para os pontos do Qdrant
ids = df['id_produto'].tolist()

# Faz o upload dos pontos para o Qdrant
print("Fazendo upload dos pontos para o Qdrant...")
client.upload_points(
    collection_name=COLLECTION_NAME,
    points=[
        models.PointStruct(
            id=ids[i],
            vector=vectors[i],
            payload=payloads[i]
        )
        for i in range(len(ids))
    ],
    wait=True # Espera a operação ser concluída
)

print("Processo concluído!")
status_colecao = client.get_collection(collection_name=COLLECTION_NAME)
print(f"Total de pontos na coleção: {status_colecao.points_count}")


In [ ]:

# --- PASSO 4: TESTE DE BUSCA ---

print("\n--- Testando a busca semântica ---")
query_text = "um bom fone para reuniões"

# Usa embed_query para um único texto de busca
query_vector = embeddings.embed_query(query_text)

# Realiza a busca na coleção
search_results = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    limit=3, # Retorna os 3 resultados mais similares
)

print(f"\nResultados da Busca por: '{query_text}'\n")
for result in search_results:
    print(f"ID do Produto: {result.id}")
    print(f"Score de Similaridade: {result.score:.4f}")
    print(f"Payload: {result.payload}")
    print("-" * 20)